In [ ]:
import pandas as pd
import numpy as np
import os
import pyxlsb
from pyxlsb import open_workbook as open_xlsb
import pyodbc, sqlalchemy
import socket
import openpyxl

def insert_data_to_sql():
    file_path = input("Enter Folder Path: ")
    sheet_name = input("Enter Sheet Name: ")
    driver_name = input("Enter Driver Name: ")
    db_name = input("Enter Database Name:")
    table_name = input("Enter Table Name:")
    dataframes = []
    print("Searching Files in: " + file_path)
    for filename in os.listdir(file_path):
        if filename.endswith('.xlsb'):
            print("Found ",filename)
            with open_xlsb(os.path.join(file_path, filename)) as wb:
                with wb.get_sheet(sheet_name) as sheet:
                    data = []
                    for row in sheet.rows():
                        data.append([item.v for item in row])
                    df = pd.DataFrame(data[1:], columns=data[0])
                    dataframes.append(df)
        elif filename.endswith('.xlsx'):
            print("Found ",filename)
            wb = openpyxl.load_workbook(os.path.join(file_path, filename))
            sheet = wb[sheet_name]
            data = []
            for row in sheet.iter_rows():
                data.append([cell.value for cell in row])
            df = pd.DataFrame(data[1:], columns=data[0])
            dataframes.append(df)
        elif filename.endswith('.csv'):
            print("Found ",filename)
            file = os.path.join(file_path, filename)
            df = pd.read_csv(file, sheet_name)
            dataframes.append(df)
        else:
            continue
    print("Consolidating all Files")        
    consolidated_df = pd.concat(dataframes)
    print("Total rows: ", consolidated_df.shape[0])
    print("Connecting to SQL")
    conn=sqlalchemy.create_engine(f'mssql+pyodbc://{socket.gethostname()}/{db_name}?trusted_connection=yes&driver={driver_name}')
    print("Loading into DB")
    consolidated_df.to_sql(table_name, con = conn, index=False, if_exists='append')
    print("Loading Completed")
    
    